# Using Autograd in TensorFlow to Solve a Regression Problem

We usually use TensorFlow to build a neural network. However, TensorFlow is not limited to this. Behind the scenes, TensorFlow is a tensor library with automatic differentiation capability. Hence you can easily use it to solve a numerical optimization problem with gradient descent, which is the algorithm to train a neural network.

In TensorFlow 2.x, you can define variables and constants as TensorFlow objects and build an expression with them. The expression is essentially a function of the variables. Hence you may derive its derivative function, i.e., the differentiation or the gradient. This feature is one of the many fundamental features in TensorFlow. The deep learning model will make use of this in the training loop.

It is easier to explain autograd with an example.

In [ ]:
import tensorflow as tf

In [3]:
# Create a constant matrix

x = tf.constant([1, 2, 3])
print(x)

print(x.shape)
print(x.dtype)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
(3,)
<dtype: 'int32'>


This creates an integer vector (in the form of Tensor object). This vector can work like a NumPy vector in most cases. For example, you can do x+x or 2*x, and the result is just what you would expect. TensorFlow comes with many functions for array manipulation that match NumPy, such as tf.transpose or tf.concat.

Creating variables in TensorFlow is just the same, for example:

In [2]:
x = tf.Variable([1, 2, 3])
print(x)
print(x.shape)
print(x.dtype)

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>
(3,)
<dtype: 'int32'>


The operations (such as x+x and 2*x) that you can apply to Tensor objects can also be applied to variables. The difference between variables and constants is that the former allows the value to change while the latter is immutable. This distinction is important when you run a gradient tape as follows:

In [4]:
x = tf.Variable(3.6)

with tf.GradientTape() as tape:
    y = x*x

dy = tape.gradient(y, x)
print(dy)

tf.Tensor(7.2, shape=(), dtype=float32)


What it does is the following: This defined a variable x (with value 3.6) and then created a gradient tape. While the gradient tape is working, it computes y=x*x or y = x2. The gradient tape monitored how the variables are manipulated. Afterward, you asked the gradient tape to find the derivative dy/dx . You know y = x^2 means dy/dx = 2x. Hence the output would give you a value of 3.6 × 2 = 7.2.

## Using Autograd for Polynomial Regression

How this feature in TensorFlow helpful?

Let’s consider a case where you have a polynomial in the form of y = f(x), and you are given several (x, y) samples. How can you recover the polynomial f(x)? One way is to assume random coefficients for the polynomial and feed in the samples (x, y). If the polynomial is found, you should see the value of y matches f(x). The closer they are, the closer your estimate is to the correct polynomial. This is indeed a numerical optimization problem such that you want to minimize the difference between y and f(x). You can use gradient descent to solve it. Let’s consider an example. You can build a polynomial f(x) = x^2 + 2^x + 3 in NumPy as follows:



In [6]:
import numpy as np 

polynomial = np.poly1d([1,2,3])
print(polynomial)

2
1 x + 2 x + 3


You may use the polynomial as a function, such as:

In [7]:
print(polynomial(1.5))

8.25


And this prints 8.25, for (1.5)^2 + 2 × (1.5) + 3 = 8.25.  Now you can generate a number of samples from this function using NumPy:

In [9]:
N = 20

# Generate random samples between -10 to +10
X = np.random.uniform(-10,10, size=(N,1))
Y = polynomial(X)

In [18]:
print(X.transpose())
print(Y.transpose())

[[-2.10820045  7.81068172  4.18338956  6.8560617   4.1062284   7.41163675
   3.56875654  1.02081277  4.98730082 -2.66548641  7.91535862  5.13490471
   4.64020175 -8.50678809  1.9533191   2.4874964  -5.12888073  9.51591432
   0.81971168 -7.65530475]]
[[  3.22810823  79.6281124   28.86752736  63.71770536  28.07356847
   72.75563275  22.87353632   6.08368426  37.84777107   4.77384499
   81.48361931  39.6370558   33.81187584  58.35186742  10.72209369
   14.16263114  19.04765609 112.58445405   5.31135062  46.29308132]]


In the above, both X and Y are NumPy arrays of the shape (20,1), and they are related as y = f(x) for the polynomial f(x).

Now, assume you do not know what the polynomial is, except it is quadratic. And you want to recover the coefficients. Since a quadratic polynomial is in the form of Ax^2 +Bx+C, you have three unknowns to find. You can find them using the gradient descent algorithm you implement or an existing gradient descent optimizer. The following demonstrates how it works:

In [34]:
# assume samples X and Y are prepared elsewhere

XX = np.hstack([X*X, X, np.ones_like(X)])

In [35]:
XX

array([[ 4.44450912, -2.10820045,  1.        ],
       [61.00674896,  7.81068172,  1.        ],
       [17.50074824,  4.18338956,  1.        ],
       [47.00558197,  6.8560617 ,  1.        ],
       [16.86111167,  4.1062284 ,  1.        ],
       [54.93235926,  7.41163675,  1.        ],
       [12.73602324,  3.56875654,  1.        ],
       [ 1.04205872,  1.02081277,  1.        ],
       [24.87316944,  4.98730082,  1.        ],
       [ 7.10481782, -2.66548641,  1.        ],
       [62.65290207,  7.91535862,  1.        ],
       [26.36724638,  5.13490471,  1.        ],
       [21.53147233,  4.64020175,  1.        ],
       [72.3654436 , -8.50678809,  1.        ],
       [ 3.8154555 ,  1.9533191 ,  1.        ],
       [ 6.18763834,  2.4874964 ,  1.        ],
       [26.30541755, -5.12888073,  1.        ],
       [90.5526254 ,  9.51591432,  1.        ],
       [ 0.67192725,  0.81971168,  1.        ],
       [58.60369083, -7.65530475,  1.        ]])

In [36]:
w = tf.Variable(tf.random.normal((3,1)))    # the 3 coefficients
x = tf.constant(XX, dtype=tf.float32)       # input sample
y = tf.constant(Y, dtype=tf.float32)        # output sample

In [37]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
print(w)

for _ in range(5000):
    with tf.GradientTape() as tape:
        y_pred = x @ w
        mse = tf.reduce_sum(tf.square(y - y_pred))
    grad = tape.gradient(mse, w)
    optimizer.apply_gradients([(grad,w)])

print(w)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.10743889],
       [ 1.1160605 ],
       [-1.3658004 ]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[1.0000061],
       [2.0000079],
       [2.9996545]], dtype=float32)>



What the above code does is the following: First, it creates a variable vector w of 3 values, namely the coefficients A,B,C. Then you create an array of shape (N, 3), in which N is the number of samples in the array X. This array has 3 columns, which are the values of x^2, x, and 1, respectively. Such an array is built from the vector X using the np.hstack() function. Similarly, we build the TensorFlow constant y from the NumPy array Y. Afterwards, you use a for-loop to run gradient descent in 1,000 iterations. In each iteration, you compute x × w. in matrix form to find Ax^2 + Bx + C and assign it to the variable y_pred. Then, compare y. and y_pred and find the mean square error. Next, derive the gradient, i.e., the rate of change of the mean square error with respect to the coefficients w. And based on this gradient, you use gradient descent to update w.

In essence, the above code is to find the coefficients w that minimizes the mean square error. Putting everything together, the following is the complete code:

In [ ]:
N = 20 # number of samples

# Generate random samples between -10 to +10
polynomial = np.poly1d([1, 2, 3])
X = np.random.uniform(-10, 10, size=(N,1))
Y = polynomial(X)

# Prepare input as an array of shape (N,3)
XX = np.hstack([X*X, X, np.ones_like(X)])

# Prepare TensorFlow objects
w = tf.Variable(tf.random.normal((3,1))) # the 3 coefficients
x = tf.constant(XX, dtype=tf.float32) # input sample
y = tf.constant(Y, dtype=tf.float32) # output sample
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
print(w)

# Run optimizer
for _ in range(1000):
    with tf.GradientTape() as tape:
        y_pred = x @ w
        mse = tf.reduce_sum(tf.square(y - y_pred))
    grad = tape.gradient(mse, w)
    optimizer.apply_gradients([(grad, w)])
    
print(w)

# Use Autograd to Solve a math puzzle

What if we had:
- A + B = 8
- C - D = 6
- A + C = 13
- B + D = 8




In [46]:
import random

A = tf.Variable(random.random())
B = tf.Variable(random.random())
C = tf.Variable(random.random())
D = tf.Variable(random.random())

# Gradient descent loop
EPOCHS = 1000
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.1)

for _ in range(EPOCHS):
    with tf.GradientTape() as tape:
        y1 = A + B - 8
        y2 = C - D - 6
        y3 = A + C - 13
        y4 = B + D - 8
        sqerr = y1*y1 + y2*y2 + y3*y3 + y4*y4
    gradA, gradB, gradC, gradD = tape.gradient(sqerr, [A, B, C, D])
    optimizer.apply_gradients([(gradA, A), (gradB, B), (gradC, C), (gradD, D)])

print(A)
print(B)
print(C)
print(D)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.5000088>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=4.499996>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=9.499989>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.4999964>


In [47]:
A = 34.23
B = 76.78
C = 93.2
D = 43.84

e1 = A + B
e2 = C - D
e3 = A + C
e4 = B + D

In [49]:
A = tf.Variable(random.random())
B = tf.Variable(random.random())
C = tf.Variable(random.random())
D = tf.Variable(random.random())

# Gradient descent loop
EPOCHS = 6000
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.1)

for _ in range(EPOCHS):
    with tf.GradientTape() as tape:
        y1 = A + B - e1
        y2 = C - D - e2
        y3 = A + C - e3
        y4 = B + D - e4
        sqerr = y1*y1 + y2*y2 + y3*y3 + y4*y4
    gradA, gradB, gradC, gradD = tape.gradient(sqerr, [A, B, C, D])
    optimizer.apply_gradients([(gradA, A), (gradB, B), (gradC, C), (gradD, D)])

print(A)
print(B)
print(C)
print(D)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=34.23036>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=76.779686>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=93.19973>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=43.840034>


The above code defines the four unknown as variables with random initial values. Then you compute the result of the four equations and compare it to the expected answer. You then sum up the squared error and ask TensorFlow to minimize it. The minimum possible square error is zero, attained when our solution exactly fits the problem.

Note the way the gradient tape is asked to produce the gradient: You ask the gradient of sqerr respective to A, B, C, and D. Hence four gradients are found. You then apply each gradient to the respective variables in each iteration. Rather than looking for the gradient in four different calls to tape.gradient(), this is required in TensorFlow because the gradient of sqerr can only be recalled once by default